<a href="https://colab.research.google.com/github/gdannay/BSQ/blob/master/BERT_new_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers

     |████████████████████████████████| 1.1MB 8.4MB/s 
     |████████████████████████████████| 1.1MB 29.5MB/s 
     |████████████████████████████████| 3.0MB 52.1MB/s 
     |████████████████████████████████| 890kB 52.5MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

df = pd.read_csv('train_cpc.csv')
df.head(10)

,group,def
0,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...
1,A01,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...
2,A01,Hand tools (edge trimmers for lawns A01G3/06 ...
3,A01,Spades; Shovels {(hand-operated dredgers E02F3...
4,A01,{Collapsible; extensible; combinations with ot...
5,A01,{Foot protectors attached to the blade}
6,A01,{with auxiliary handles for facilitating lifting}
7,A01,{with ground abutment shoes or earth anchors f...
8,A01,with teeth
9,A01,Hoes; Hand cultivators {(rakes A01D7/00; forks...


In [ ]:
group = [ele.split(",")[0] for ele in list(df["group"])]
df["group"] = group
df["title"] = df["def"]
print(df.head())
print(df["group"].value_counts())

  group  ...                                              title
0   A01  ...  AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...
1   A01  ...  SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...
2   A01  ...  Hand tools (edge trimmers for lawns A01G3/06  ...
3   A01  ...  Spades; Shovels {(hand-operated dredgers E02F3...
4   A01  ...  {Collapsible; extensible; combinations with ot...

[5 rows x 3 columns]
H01    20225
Y10    16052
A61    12962
C12    10332
G01     9840
       ...  
C99        3
E99        3
A99        3
F99        3
H99        3
Name: group, Length: 136, dtype: int64


In [ ]:
possible_labels = df.group.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'A01': 0,
 'A21': 1,
 'A22': 2,
 'A23': 3,
 'A24': 4,
 'A41': 5,
 'A42': 6,
 'A43': 7,
 'A44': 8,
 'A45': 9,
 'A46': 10,
 'A47': 11,
 'A61': 12,
 'A62': 13,
 'A63': 14,
 'A99': 15,
 'B01': 16,
 'B02': 17,
 'B03': 18,
 'B04': 19,
 'B05': 20,
 'B06': 21,
 'B07': 22,
 'B08': 23,
 'B09': 24,
 'B21': 25,
 'B22': 26,
 'B23': 27,
 'B24': 28,
 'B25': 29,
 'B26': 30,
 'B27': 31,
 'B28': 32,
 'B29': 33,
 'B30': 34,
 'B31': 35,
 'B32': 36,
 'B33': 37,
 'B41': 38,
 'B42': 39,
 'B43': 40,
 'B44': 41,
 'B60': 42,
 'B61': 43,
 'B62': 44,
 'B63': 45,
 'B64': 46,
 'B65': 47,
 'B66': 48,
 'B67': 49,
 'B68': 50,
 'B81': 51,
 'B82': 52,
 'B99': 53,
 'C01': 54,
 'C02': 55,
 'C03': 56,
 'C04': 57,
 'C05': 58,
 'C06': 59,
 'C07': 60,
 'C08': 61,
 'C09': 62,
 'C10': 63,
 'C11': 64,
 'C12': 65,
 'C13': 66,
 'C14': 67,
 'C21': 68,
 'C22': 69,
 'C23': 70,
 'C25': 71,
 'C30': 72,
 'C40': 73,
 'C99': 74,
 'D01': 75,
 'D02': 76,
 'D03': 77,
 'D04': 78,
 'D05': 79,
 'D06': 80,
 'D07': 81,
 'D10': 82,
 'D21': 83,
 '

In [ ]:
df['label'] = df.group.replace(label_dict)

In [ ]:
df.head()

,group,def,title,label
0,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,0
1,A01,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,0
2,A01,Hand tools (edge trimmers for lawns A01G3/06 ...,Hand tools (edge trimmers for lawns A01G3/06 ...,0
3,A01,Spades; Shovels {(hand-operated dredgers E02F3...,Spades; Shovels {(hand-operated dredgers E02F3...,0
4,A01,{Collapsible; extensible; combinations with ot...,{Collapsible; extensible; combinations with ot...,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['group', 'label', 'data_type']).count()
print(df.title)

0         AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...
1         SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...
2         Hand tools (edge trimmers for lawns A01G3/06  ...
3         Spades; Shovels {(hand-operated dredgers E02F3...
4         {Collapsible; extensible; combinations with ot...
                                ...                        
259653                        including tool pot or adapter
259654                                      Indexing matrix
259655                                          Rotary disc
259656                                        Chain or belt
259657                                        Miscellaneous
Name: title, Length: 259658, dtype: object


In [ ]:
for i in df[df.data_type=='val'].title.values:
  if not isinstance(i, str):
    print(i)
print(len(df[df.data_type=='val'].title.values))


38949


In [ ]:
df = df.dropna()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].title.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].title.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-large-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 1

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import tensorflow as tf # Get the GPU device name.
device_name = tf.test.gpu_device_name() # The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
  
import torch# If there's a GPU available...
if torch.cuda.is_available():        # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

In [ ]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/content/gdrive/My Drive/Bert Patent/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

Buffered data was truncated after reaching the output size limit.

In [ ]:
import numpy as np

model = BertForSequenceClassification.from_pretrained("bert-large-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('/content/gdrive/My Drive/Bert Patent/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Class: A01
Accuracy: 198/425

Class: A21
Accuracy: 0/44

Class: A22
Accuracy: 7/31

Class: A23
Accuracy: 166/299

Class: A24
Accuracy: 13/51

Class: A41
Accuracy: 18/97

Class: A42
Accuracy: 0/14

Class: A43
Accuracy: 59/107

Class: A44
Accuracy: 0/61

Class: A45
Accuracy: 18/136

Class: A46
Accuracy: 12/37

Class: A47
Accuracy: 281/551

Class: A61
Accuracy: 1248/1938

Class: A62
Accuracy: 0/47

Class: A63
Accuracy: 249/527

Class: B01
Accuracy: 519/1041

Class: B02
Accuracy: 0/54

Class: B03
Accuracy: 0/57

Class: B04
Accuracy: 0/16

Class: B05
Accuracy: 85/191

Class: B06
Accuracy: 0/6

Class: B07
Accuracy: 0/35

Class: B08
Accuracy: 0/24

Class: B09
Accuracy: 0/7

Class: B21
Accuracy: 87/253

Class: B22
Accuracy: 26/137

Class: B23
Accuracy: 324/622

Class: B24
Accuracy: 27/97

Class: B25
Accuracy: 7/154

Class: B26
Accuracy: 3/90

Class: B27
Accuracy: 13/85

Class: B28
Accuracy: 29/108

Class: B29
Accuracy: 523/960

Class: B30
Accuracy: 0/36

Class: B31
Accuracy: 1/100

Class: B32


In [ ]:
    
val_loss, predictions, true_vals = evaluate(dataloader_validation)
val_f1 = f1_score_func(predictions, true_vals)
print(f'Validation loss: {val_loss}')
print(f'F1 Score (Weighted): {val_f1}')

Validation loss: 2.0055829318375475
F1 Score (Weighted): 0.5286604693915155


In [9]:
!git init
!git config --global user.email "gregory.dannay@sciencespo.fr”
!git config --global user.name "gdannay"
!git add -A
!git commit -m "first commit"
#!git remote add origin https://github.com/LongRunGrowth/Technology_classification.git


Reinitialized existing Git repository in /content/.git/
/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file
On branch master
nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address


In [11]:
!git remote add origin https://github.com/LongRunGrowth/Technology_classification.git

fatal: remote origin already exists.


In [12]:
!git push -u origin master

fatal: could not read Username for 'https://github.com': No such device or address
